In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("deepcontractor/supreme-court-judgment-prediction")

print("Path to dataset files:", path)

In [ ]:
import os

# Path to your dataset
dataset_path = "/root/.cache/kagglehub/datasets/deepcontractor/supreme-court-judgment-prediction/versions/1"

# List all files and folders
for root, dirs, files in os.walk(dataset_path):
    print(f"\n📁 Directory: {root}")
    for file in files:
        print(f"   📄 {file}")


In [ ]:
import pandas as pd

# Define the file path
file_path = "/root/.cache/kagglehub/datasets/deepcontractor/supreme-court-judgment-prediction/versions/1/justice.csv"

# Load the CSV into a DataFrame
df = pd.read_csv(file_path)

# Show the first 5 rows
df.dropna(inplace=True)
df.head()

In [ ]:
df1 = df.copy()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df1.drop(columns=['Unnamed: 0', 'docket','name','first_party','second_party', 'issue_area',
                 'facts_len', 'majority_vote', 'minority_vote', 'href', 'ID','term'], inplace=True)

In [ ]:
df_cat = df1[['decision_type', 'disposition']]

In [ ]:
df_target = df1['first_party_winner']
df_nlp = df1['facts']

In [ ]:
df_cat.reset_index(drop=True, inplace=True)
df_target.reset_index(drop=True, inplace=True)
df_nlp.reset_index(drop=True, inplace=True)
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df_target= label_encoder.fit_transform(df_target)
df_target1 = pd.DataFrame(df_target, columns=['first_party_winner'])
df_target1

In [ ]:
frames = [df_cat, df_target1]
df_concat = pd.concat(frames, axis=1, join='inner')
df_concat

In [ ]:
df_nlp1 = pd.DataFrame(df_nlp, columns=['facts'])
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'<[^<>]*>', '', regex=True)
df_nlp1

In [ ]:
corpus = df_nlp1["facts"]

import nltk
# Removed the download for punkt_tab as it was already downloaded in the previous run
nltk.download('stopwords')
nltk.download('wordnet')

ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()
lst_stopwords = nltk.corpus.stopwords.words("english")
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in
                    lst_stopwords]

    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## back to string from list
    text = " ".join(lst_text)
    return text
df_nlp1["facts_clean"] = df_nlp1["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
df_nlp1

In [ ]:
df_nlp2 = pd.concat([df_nlp1,df_target1['first_party_winner']],axis=1, join='inner')
df_nlp2["first_party_winner"].value_counts()

In [ ]:
# Separate the classes
class_1 = df_nlp2[df_nlp2["first_party_winner"] == 1]
class_0 = df_nlp2[df_nlp2["first_party_winner"] == 0]

# Undersample class 1 to match count of class 0 (1031)
class_1_under = class_1.sample(len(class_0), random_state=42)

# Combine the two classes
df_nlp2 = pd.concat([class_1_under, class_0], axis=0)

# Shuffle the combined DataFrame
df_nlp2 = df_nlp2.sample(frac=1, random_state=42).reset_index(drop=True)

# Confirm balanced data
print(df_nlp2["first_party_winner"].value_counts())


In [ ]:
df1 = df_nlp2.copy()

In [ ]:
df1

In [ ]:
df1.drop(columns=['facts'], inplace=True)

In [ ]:
df1

In [ ]:
from sklearn.model_selection import train_test_split

# Convert labels to binary: True → 1, False → 0
df1['label'] = df1['first_party_winner'].astype(int)
df1['text'] = df1['facts_clean']

# Split into train/test
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df1['text'].tolist(), df1['label'].tolist(), test_size=0.2, random_state=42
)


In [ ]:
from collections import Counter

# Count in training labels
train_distribution = Counter(train_labels)
print("Training Label Distribution:", train_distribution)

# Count in validation labels
val_distribution = Counter(val_labels)
print("Validation Label Distribution:", val_distribution)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
import torch

# ✅ Load better tokenizer: Legal RoBERTa
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Tokenize text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# ✅ Define Dataset Class
class CourtDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Dataset objects
train_dataset = CourtDataset(train_encodings, train_labels)
val_dataset = CourtDataset(val_encodings, val_labels)

# ✅ Define Evaluation Metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = torch.argmax(torch.tensor(logits), dim=-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds)
    }

# ✅ Load Legal-RoBERTa Model
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=2)

# ✅ Define Better Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

# ✅ Trainer with metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# 🏋️ Start Training
trainer.train()

In [ ]:
from torch.nn.functional import softmax

def predict_outcome(text, model, tokenizer):
    # Clean the text
    text_clean = utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords)

    # Tokenize
    inputs = tokenizer(text_clean, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Move to device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = softmax(logits, dim=-1)
        predicted_class = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][predicted_class].item()

    return predicted_class, confidence


In [ ]:
from transformers import pipeline

def suggest_legal_actions(facts):
    prompt = f"""You are an expert Indian legal advisor. Carefully study the case facts below, and suggest **three concrete legal actions** the First Party should take to improve their chances of winning in court. Be specific and legally sound.

Case Facts:
{facts}

Suggestions:"""

    # Load the FLAN-T5 model
    suggestions_generator = pipeline(
        "text2text-generation",
        model="google/flan-t5-large",
        device=0  # use -1 if you're on CPU
    )

    # Generate suggestion
    result = suggestions_generator(
        prompt,
        max_length=256,
        do_sample=True,
        temperature=0.8,
        top_p=0.9,
        repetition_penalty=1.2,
        num_return_sequences=1
    )

    # Extract and return suggestions
    suggestions = result[0]['generated_text'].strip()
    return suggestions


In [ ]:
input_facts = "i killed my wife"

# ✅ Call the function
output = suggest_legal_actions(input_facts)

# 📤 Print the output
print("🔎 Suggested Legal Actions:\n", output)

In [ ]:
def suggest_punishment_article(facts):
    prompt = f"""You are a legal expert. Given the following facts from a Supreme Court case, what is the most likely Article or IPC section the first party may be punished under if they lose?

Case Facts:
{facts}

Answer:"""

    # Use a language model (e.g., GPT)
    from transformers import pipeline
    article_generator = pipeline("text-generation", model="gpt2")  # Replace with better legal model if available

    result = article_generator(prompt, max_length=100, do_sample=True)[0]['generated_text']
    return result


In [ ]:
def legal_superbot(facts, model, tokenizer):
    print("🔍 Predicting outcome...")
    outcome, confidence = predict_outcome(facts, model, tokenizer)

    print("\n🛡️ Suggestions to help your case:")
    suggestions = suggest_legal_actions(facts)

    print("\n🚨 If case is lost, possible punishment under:")
    article = suggest_punishment_article(facts)

    # Format confidence as %
    confidence_percent = round(confidence * 100, 2)

    # Outcome message with confidence
    outcome_msg = (
        f"First party WINS ✅ (Confidence: {confidence_percent}%)"
        if outcome == 1 else
        f"First party LOSES ❌ (Confidence: {confidence_percent}%)"
    )

    result = {
        "Outcome": outcome_msg,
        "Suggestions": suggestions,
        "Likely Article/IPC if Lost": article
    }
    return result


In [ ]:
test_facts = """
The petitioner, a government employee, was found guilty of accepting a bribe of ₹50,000 in exchange for approving a tender. The CBI filed a chargesheet, and departmental proceedings were initiated.
"""


In [ ]:
result = legal_superbot(test_facts, model, tokenizer)

# Display nicely
for key, value in result.items():
    print(f"\n🔹 {key}:")
    print(value)

In [ ]:
# ✅ Save model and tokenizer
trainer.save_model("legalbert_model")  # Saves model
tokenizer.save_pretrained("legalbert_model")  # Saves tokenizer


In [ ]:
# ✅ Save model and tokenizer
trainer.save_model("legalbert_model")  # Saves model
tokenizer.save_pretrained("legalbert_model")  # Saves tokenizer


In [ ]:
!zip -r legalbert_model.zip legalbert_model/
from google.colab import files
files.download("legalbert_model.zip")

In [ ]:
# ✅ Install transformers (if not already)
!pip install transformers --upgrade -q
!pip install accelerate --upgrade -q

# ✅ Import Required Modules
from transformers import pipeline
import torch

# ✅ Load the Legal GPT-2 Style Model
generator = pipeline(
    "text-generation",
    model="SkolkovoInstitute/rugpt3small_based_on_gpt2",
    tokenizer="SkolkovoInstitute/rugpt3small_based_on_gpt2",
    device=0 if torch.cuda.is_available() else -1
)

# ✅ Define Prompt
facts = """
The defendant failed to deliver the shipment of machinery on the agreed date despite multiple reminders. The first party suffered financial losses due to project delays.
"""

prompt = f"""You are a legal assistant. Given the case facts below, suggest the best possible actions the first party can take to improve their chances of winning in court.

Case Facts:
{facts}

Suggestions:"""

# ✅ Generate Suggestions
output = generator(prompt, max_length=200, do_sample=True, temperature=0.8)[0]["generated_text"]

# ✅ Extract and Display Suggestions
suggestions = output.split("Suggestions:")[-1].strip()
print("📌 Suggestions for First Party:\n")
print(suggestions)


In [ ]:
model.save_pretrained("./my_model/")
tokenizer.save_pretrained("./my_model/")

In [ ]:
def legal_superbot(facts, model, tokenizer):
    print("🔍 Predicting outcome...")
    outcome, confidence = predict_outcome(facts, model, tokenizer)

    print("\n🛡️ Suggestions to help your case:")
    suggestions = suggest_legal_actions(facts)

    print("\n🚨 If case is lost, possible punishment under:")
    article = suggest_punishment_article(facts)

    # Format confidence as %
    confidence_percent = round(confidence * 100, 2)

    # Outcome message with confidence
    outcome_msg = (
        f"First party WINS ✅ (Confidence: {confidence_percent}%)"
        if outcome == 1 else
        f"First party LOSES ❌ (Confidence: {confidence_percent}%)"
    )

    result = {
        "Outcome": outcome_msg,
        "Suggestions": suggestions,
        "Likely Article/IPC if Lost": article
    }
    return result